# Fundamentos de BD (Márcio Victorino)

## Partes do Projeto:
- Introdução.
- Modelo de dados Relacional.
- O script SQL que gerou o banco de dados.
- Apresentar o processo de ETL (Extract, Transform, Load) para importação dos
dados para o banco de dados.
- Utilização de pelo menos uma View.
- Utilização de pelo menos uma Procedure (com comandos condicionais).
- Utilização de pelo menos um trigger (com comandos condicionais).
- No mínimo 5 Consultas SQL (a complexidade da consulta será avaliada).

**Data Limite para a Entrega da Parte Escrita: 02/06/2023**

## Observações

- Dados abertos (ex: FNDE, bolsa família)
- Linkar com o Artigo da matéria do Ladeira MD
- Joga em um CSV e alimenta o BD SQL
- No mínimo milhares de dezenas de linhas
- Não executar a consulta na apresentação
- Printar os resultados e mostrar na apresentação

## Alunos: 

Marcelo Anselmo de Souza Filho
- Matrícula: **231109719**
- Email: **marcelofilho@mpf.mp.br**

Arivaldo Gonçalves
- Matrícula: **111**
- Email: **arivaldofreitas@correios.com.br**

Luciana Maria
- Matrícula: **1111**
- Email: **luciana@mpdft.mp.br**

____

In [1]:
# !pip install mysql-connector-python

# Carregar dados

In [2]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet(
    "../output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip")

qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)


FileNotFoundError: [Errno 2] No such file or directory: './output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip'

# Criando DB

https://dev.mysql.com/doc/connector-python/en/connector-python-example-ddl.html

In [ ]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode

DB_NAME = 'test'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE `employees` ("
    "  `emp_no` int(11) NOT NULL AUTO_INCREMENT,"
    "  `birth_date` date NOT NULL,"
    "  `first_name` varchar(14) NOT NULL,"
    "  `last_name` varchar(16) NOT NULL,"
    "  `gender` enum('M','F') NOT NULL,"
    "  `hire_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE `departments` ("
    "  `dept_no` char(4) NOT NULL,"
    "  `dept_name` varchar(40) NOT NULL,"
    "  PRIMARY KEY (`dept_no`), UNIQUE KEY `dept_name` (`dept_name`)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE `salaries` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `salary` int(11) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`,`from_date`), KEY `emp_no` (`emp_no`),"
    "  CONSTRAINT `salaries_ibfk_1` FOREIGN KEY (`emp_no`) "
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE `dept_emp` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `dept_no` char(4) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`,`dept_no`), KEY `emp_no` (`emp_no`),"
    "  KEY `dept_no` (`dept_no`),"
    "  CONSTRAINT `dept_emp_ibfk_1` FOREIGN KEY (`emp_no`) "
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE,"
    "  CONSTRAINT `dept_emp_ibfk_2` FOREIGN KEY (`dept_no`) "
    "     REFERENCES `departments` (`dept_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE `dept_manager` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `dept_no` char(4) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date NOT NULL,"
    "  PRIMARY KEY (`emp_no`,`dept_no`),"
    "  KEY `emp_no` (`emp_no`),"
    "  KEY `dept_no` (`dept_no`),"
    "  CONSTRAINT `dept_manager_ibfk_1` FOREIGN KEY (`emp_no`) "
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE,"
    "  CONSTRAINT `dept_manager_ibfk_2` FOREIGN KEY (`dept_no`) "
    "     REFERENCES `departments` (`dept_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE `titles` ("
    "  `emp_no` int(11) NOT NULL,"
    "  `title` varchar(50) NOT NULL,"
    "  `from_date` date NOT NULL,"
    "  `to_date` date DEFAULT NULL,"
    "  PRIMARY KEY (`emp_no`,`title`,`from_date`), KEY `emp_no` (`emp_no`),"
    "  CONSTRAINT `titles_ibfk_1` FOREIGN KEY (`emp_no`)"
    "     REFERENCES `employees` (`emp_no`) ON DELETE CASCADE"
    ") ENGINE=InnoDB")


In [ ]:
cnx = mysql.connector.connect(user='test', password='test')
cursor = cnx.cursor()


In [ ]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)


try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)


In [ ]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()


Creating table employees: OK
Creating table departments: OK
Creating table salaries: OK
Creating table dept_emp: OK
Creating table dept_manager: OK
Creating table titles: OK


# Inserindo Dados

In [ ]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)


df = pd.read_parquet(
    f"../output/silver/rais_TODOS_ANOS_TODAS_COLS_RAIS_VINC_PUB.parquet.gzip")

qnt_total = len(df)

print(f"""
Quantidade docs: {qnt_total}
""")

df.head(2)



Quantidade docs: 384175



,Bairros SP,Bairros Fortaleza,Bairros RJ,Causa Afastamento 1,Causa Afastamento 2,Causa Afastamento 3,Motivo Desligamento,ocupacao_id,CNAE 2.0 Classe,CNAE 95 Classe,Distritos SP,Vínculo Ativo 31/12,Faixa Etária,Faixa Hora Contrat,Faixa Remun Dezem (SM),Faixa Remun Média (SM),Faixa Tempo Emprego,Escolaridade após 2005,Qtd Hora Contr,idade,Ind CEI Vinculado,Ind Simples,Mês Admissão,Mês Desligamento,Mun Trab,id_uf_municipio,Nacionalidade,Natureza Jurídica,Ind Portador Defic,Qtd Dias Afastamento,raca_cor_id,Regiões Adm DF,Vl Remun Dezembro Nom,Vl Remun Dezembro (SM),remuneracao_media,Vl Remun Média (SM),CNAE 2.0 Subclasse,sexo_id,Tamanho Estabelecimento,Tempo Emprego,Tipo Admissão,Tipo Estab,Tipo Estab.1,Tipo Defic,Tipo Vínculo,IBGE Subsetor,Vl Rem Janeiro CC,Vl Rem Fevereiro CC,Vl Rem Março CC,Vl Rem Abril CC,Vl Rem Maio CC,Vl Rem Junho CC,Vl Rem Julho CC,Vl Rem Agosto CC,Vl Rem Setembro CC,Vl Rem Outubro CC,Vl Rem Novembro CC,Ano Chegada Brasil,Ind Trab Intermitente,Ind Trab Parcial,ano,sigla_uf,sigla_municipio,Vl Rem Janeiro SC,Vl Rem Fevereiro SC,Vl Rem Março SC,Vl Rem Abril SC,Vl Rem Maio SC,Vl Rem Junho SC,Vl Rem Julho SC,Vl Rem Agosto SC,Vl Rem Setembro SC,Vl Rem Outubro SC,Vl Rem Novembro SC
0,{ñ class},{ñ class},{ñ class},99,99,99,12,317210,47512,52450,{ñ class},0,2,6,0,1,1,6,44,17,0,1,8,10,0,500240,10,2135,0,0,2,0,0.0,0.0,997.99,0.99,4751201,1,1,2.9,1,1,CNPJ,0,10,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.98,9.98,0.0,0.0,0,0,0,2019,MS,Caarapo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{ñ class},{ñ class},{ñ class},80,99,99,11,212420,10112,15113,{ñ class},0,5,6,0,99,8,6,44,36,0,0,0,01,500570,500570,10,2046,0,1,2,0,0.0,0.0,0.00,0.00,1011201,1,10,170.2,0,1,CNPJ,0,10,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0,0,0,2019,MS,Navirai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from __future__ import print_function
from datetime import date, datetime, timedelta
import mysql.connector

cnx = mysql.connector.connect(user='test', password='test', database='test')
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
                "(first_name, last_name, hire_date, gender, birth_date) "
                "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))

# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

# Insert salary information
data_salary = {
    'emp_no': emp_no,
    'salary': 50000,
    'from_date': tomorrow,
    'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()
